In [1]:
import math

In [2]:
def snafu_chr_to_digit(char: str) -> int:
    return {
        "2": 2,
        "1": 1,
        "0": 0,
        "-": -1,
        "=": -2,
    }[char]


def digit_to_snafu_chr(digit: int) -> int:
    return {
        -2: "=",
        -1: "-",
        0: "0",
        1: "1",
        2: "2",
    }[digit]


def snafu_to_decimal(snafu: str) -> int:
    return sum(
        5**chr_idx * snafu_chr_to_digit(char)
        for chr_idx, char in enumerate(list(snafu)[::-1])
    )


def decimal_to_snafu(decimal: int) -> str:
    if decimal == 0:
        return ""
    snafu_len = math.ceil(math.log(decimal + 1, 5)) + 1
    snafu = [0] * snafu_len
    def _snafu_carry(val: int) -> tuple[int, int]:
        val = ((val + 2) % 5) - 2
        return val, 1 if val < 0 else 0

    for idx in range(snafu_len - 1):
        val = (decimal // 5**idx) % 5
        val, snafu_carry = _snafu_carry(val)
        snafu[idx] += val
        snafu[idx + 1] += snafu_carry
        for carry_idx in range(idx, snafu_len - 1):
            if snafu[carry_idx] <= 2:
                break
            snafu_val, snafu_carry = _snafu_carry(snafu[carry_idx])
            if snafu_carry:
                snafu[carry_idx] = snafu_val
                snafu[carry_idx + 1] += snafu_carry
    return strip_leading_zeros("".join(map(digit_to_snafu_chr, snafu[::-1])))


def strip_leading_zeros(val: str) -> str:
    while val:
        if val[0] == "0":
            val = val[1:]
        else:
            break
    return val


for idx in range(0, 1000):
    snafu_val = decimal_to_snafu(idx)
    reconstr = snafu_to_decimal(snafu_val)
    assert reconstr == idx, f"Failed at {idx}"

In [3]:
def parse_args(filename: str) -> None:
    with open(filename) as f:
        snafu_sum = sum(snafu_to_decimal(row.rstrip()) for row in f)
    return decimal_to_snafu(snafu_sum)

In [4]:
print(parse_args("test-input.txt"))

2=-1=0


In [5]:
print(parse_args("input.txt"))

20===-20-020=0001-02
